# Langton's Ant

Wikipedia url : https://en.wikipedia.org/wiki/Langton's_ant

## Rules

> At a white square, turn 90° clockwise, flip the color of the square, move forward one unit

> At a black square, turn 90° counter-clockwise, flip the color of the square, move forward one unit

## Liberaires

In [1]:
import pygame
import numpy as np

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Parameters and colors

In [2]:
## Dimensions of the grid
w  = 800  # the width of the grid
h  = 500  # the height of the grid
rw = 5    # the width of the cells of the grid
rh = 5    # the height of the cells of the grid

## Colors
GREEN = (34,139,34)
WHITE = (255,255,255)
GRAY = (128,128,128)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
BLUE = (28,128,130)

## Ant Class

In [11]:
class Grid(object):
    
    def __init__(self, w:int=w, h:int=h, rw:int=rw, rh:int=rh, c:tuple=GRAY):
        self.w = w
        self.h = h
        self.rw = rw
        self.rh = rh
        self.color = c
        self.create_grid()
        
    def create_grid(self):
        self.grid = np.zeros((self.h // self.rh, self.w // self.rw)).astype('int')
        
    def draw_grid(self, screen:pygame.display.set_mode):
        """draw a grid on the screen with the given dimensions"""
        
        ## draw the vertical lines
        for i in range(self.w // self.rw + 1):
            pygame.draw.lines(screen, self.color, False, [(i * self.rh, 0), (i * self.rh, self.h)], 1)

        ## draw the horizontal lines
        for i in range(h // rh + 1):
            pygame.draw.lines(screen, self.color, False, [(0, i * self.rw), (self.w, i * self.rw)], 1)
            
    def draw_cell(self, screen:pygame.display.set_mode, i, j):
        #for i in range(len(self.grid)):
           # for j in range(len(self.grid[0])):
                if self.grid[i, j] == 1:
                    cell_color = BLACK
                else:
                    cell_color = WHITE
                pygame.draw.rect(screen, cell_color, (j*self.rw + 1, i*self.rh + 1, rw - 1, rh - 1))

In [12]:
class Ant(object):
    
    """The Ant Class
    x  : required : the x coodinate of the ant
    y  : required : the y coodinate of the ant
    vx : optional : the x veloctiy of the ant
    vy : optional : the y velocity of the ant
    c  : optional : the color of the ant

"""
    
    def __init__(self, x:int, y:int, vx:int=0, vy:int=1, c:tuple=RED):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.color = c
        self.step = 0
        
    def update_cell(self, grid:Grid):
        """update the color of the cell on which the ant is standing"""
        
        cell = grid.grid[int(self.y // grid.rh) % grid.grid.shape[0], int(self.x // grid.rw) % grid.grid.shape[1]]
        
        if cell == 1:
            if self.vx != 0 and self.vy == 0:
                self.vy = -self.vx
                self.vx = 0
            elif self.vy != 0 and self.vx == 0:
                self.vx = self.vy
                self.vy = 0
            cell = 0
            
        else:
            if self.vx != 0 and self.vy == 0:
                self.vy = self.vx
                self.vx = 0
            elif self.vy != 0 and self.vx == 0:
                self.vx = -self.vy
                self.vy = 0
            cell = 1
            
        grid.grid[int(self.y // grid.rh) % grid.grid.shape[0], int(self.x // grid.rw) % grid.grid.shape[1]] = cell
        
        
    def update(self, grid:Grid):
        """update the postion of the ant"""
        
        self.x += self.vx * grid.rw
        self.y += self.vy * grid.rh
        self.step += 1
        
    def draw(self, screen:pygame.display.set_mode):
        
        """draw the ant on the screen"""
        self.x = rw * (self.x // rw)
        self.y = rh * (self.y // rh)
        pygame.draw.rect(screen, self.color, (self.x + 1, self.y + 1, rw - 1, rh - 1))

In [35]:
def main():
    pygame.init()
    screen = pygame.display.set_mode((w, h + 50))
    pygame.display.set_caption('Ants')
    screen.fill(WHITE)
    clock = pygame.time.Clock()
    ant = Ant(x=w//2- rw, y=h//2 - rh)
    grid = Grid()
    grid.draw_grid(screen)
    ant.draw(screen)
    start_text = "press SPACE to start"
    pause_text = "press p to pause"
    quit_text  = "press q to quit"
    counter_text = f'step : {ant.step}'
    
    font = pygame.font.SysFont('Arial', 18)
    text1 = font.render(start_text, True, BLACK)
    text2 = font.render(pause_text, True, BLACK)
    text3 = font.render(quit_text, True, BLACK)
    text4 = font.render(counter_text, True, BLACK)
    
    screen.blit(text1, (10, h + 2))
    screen.blit(text2, (w // 3, h + 2))
    screen.blit(text3, (2* (w // 3)-30, h + 2))
    screen.blit(text4, (w - 120, h + 2))
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONUP:
                point = pygame.mouse.get_pos()
                (i, j) = (point[1] // grid.rw, point[0] // grid.rh)
                grid.grid[i, j] = 1
                grid.draw_cell(screen, i, j)
                ant.draw(screen)
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    simulate = True
                    while simulate:
                        ant.update_cell(grid)
                        (i, j) = int(ant.y // grid.rw) % grid.grid.shape[0], int(ant.x // grid.rh) % grid.grid.shape[1]
                        grid.draw_cell(screen, i, j)
                        ant.update(grid)
                        ant.draw(screen)
                        counter_text = f'step : {ant.step}'
                        text4 = font.render(counter_text, True, BLACK)
                        pygame.draw.rect(screen, WHITE, (w-120, h+1, 120, 50))
                        screen.blit(text4, (w - 120, h + 2))
                        pygame.display.update()
                        for event in pygame.event.get():
                            if event.type == pygame.KEYDOWN:
                                if event.key == pygame.K_q:
                                    simulate = False
                                    running = False
                                if event.key == pygame.K_SPACE:
                                    simulate = not simulate
        pygame.display.update()

    pygame.quit()
                

In [36]:
main()

In [7]:
def main():
    screen = pygame.display.set_mode((w, h))
    pygame.display.set_caption("Langton's ant")
    screen.fill(WHITE)
    clock = pygame.time.Clock()
    ant = Ant(x=w//2- rw, y=h//2 - rh)
    grid = Grid()
    grid.draw_grid(screen)
    ant.draw(screen)
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONUP:
                point = pygame.mouse.get_pos()
                (i, j) = (point[0] // grid.rw, point[1] // grid.rh)
                grid.grid[i, j] = 1
                grid.draw_cells(screen)
                ant.draw(screen)
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_s:
                    simulate = True
                    while simulate:
                        ant.update(grid)
                        grid.draw_cells(screen)
                        ant.draw(screen)
                        pygame.display.update()
        pygame.display.update()

    pygame.quit()
                

In [8]:
if __name__ == '__main__':
    main()